<a href="https://colab.research.google.com/github/Balisa50/restaurant-data-analysis/blob/main/Geospatial_mapping_and_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Read and print the dataset
link ="https://raw.githubusercontent.com/Oyeniran20/axia_class_cohort_7/refs/heads/main/Dataset%20.csv"
df = pd.read_csv(link)


In [ ]:
!pip install geopandas shapely

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import zipfile
import os
from shapely.geometry import Point
from google.colab import files

# Map Restaurants by Rating


#  Upload ZIP file

uploaded = files.upload()
for filename in uploaded:
    if filename.endswith(".zip"):
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall("shapefile_data")
        print(f"Unzipped {filename} into 'shapefile_data' folder.")
        break

# Check if data got Longitude, Latitude, and Rating
if not {"Longitude", "Latitude", "Aggregate rating"}.issubset(df.columns):
    print("Error: Dataset missing important column (Longitude, Latitude, or Rating).")
else:
    #  Convert to map points
    df["geometry"] = [Point(xy) for xy in zip(df["Longitude"], df["Latitude"])]
    gdf = gpd.GeoDataFrame(df, geometry="geometry")

    #  Find the .shp file in the folder
    shapefile_path = None
    for file in os.listdir("shapefile_data"):
        if file.endswith(".shp"):
            shapefile_path = os.path.join("shapefile_data", file)
            break

    if shapefile_path:
        try:
            #  Load the map (shapefile)
            world = gpd.read_file(shapefile_path)
        except FileNotFoundError:
            print("Error: Shapefile not found, maybe check ZIP file?")
        except ValueError:
            print("Error: Can’t read shapefile, something wrong.")
        else:
            #  Make map, focus on restaurant area
            center_lat, center_lon = gdf["Latitude"].mean(), gdf["Longitude"].mean()
            my_map = folium.Map(location=[center_lat, center_lon], zoom_start=4, tiles="CartoDB positron")

            # Add country shape to map
            folium.GeoJson(
                world,
                style_function=lambda x: {
                    "fillColor": "lightblue",
                    "color": "black",
                    "weight": 0.5,
                    "fillOpacity": 0.4,
                },
            ).add_to(my_map)

            #  Function to set marker color based on rating
            def get_color(rating):
                if rating >= 4.5:
                    return "green"  # Very good
                elif 3.0 <= rating < 4.5:
                    return "yellow"  # Okay
                else:
                    return "red"  # Not good

            #  Add restaurants to map
            for _, row in gdf.iterrows():
                name = row.get("Restaurant Name", row.get("Restaurant", "Unknown"))
                rating = row.get("Aggregate rating", 0)  # If no rating, assume 0
                color = get_color(rating)

                folium.CircleMarker(
                    location=[row["Latitude"], row["Longitude"]],
                    radius=5,
                    color=color,
                    fill=True,
                    fill_color=color,
                    fill_opacity=0.7,
                    popup=f"Restaurant: {name}<br>Rating: {rating}",
                ).add_to(my_map)

            #  Show map
            display(my_map)

            # Find correlation (does location affect rating?)
            correlation = df[["Longitude", "Latitude", "Aggregate rating"]].corr()
            print("Correlation Matrix:\n", correlation)

    else:
        print("No .shp file found in the folder. Try again?")

